In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import time
import logging
from multiprocessing import Pool, cpu_count
import traceback

import sys
sys.path.append('../python')

import utils
import examples
from optimal import OptimalStrategy
from nominal import NominalStrategy
from ofu import OFUStrategy
from sls import SLS_FIRStrategy, SLS_CommonLyapunovStrategy, sls_common_lyapunov, SLSInfeasibleException
from ts import TSStrategy
from lspi import LSPIStrategy
from mflq import MFLQStrategy

import matplotlib.pylab as plt
import seaborn as sns
sns.set_style('ticks')

logging.basicConfig(level=logging.DEBUG)

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [8]:
# PARAMETERS
rng = np.random
horizon = 10000
trials_per_method = 10

## Defining True System Dynamics

In [9]:
def set_up_example(A_star, B_star, qr_ratio, prime_horizon, prime_excitation, sigma_excitation=0.1):
    n,p = B_star.shape
    # design a stabilizing controller
    _, K_init = utils.dlqr(A_star, B_star, 1e-3*np.eye(n), np.eye(p))
    assert utils.spectral_radius(A_star + B_star.dot(K_init)) < 1
    Q = qr_ratio * np.eye(n)
    R = np.eye(p)
    sigma_w = 1
    return A_star, B_star, K_init, Q, R, prime_horizon, prime_excitation, sigma_excitation, sigma_w

def laplacian_dynamics(qr_ratio=1e1, prime_horizon=100, prime_excitation=1):
    A_star, B_star = examples.unstable_laplacian_dynamics()
    return set_up_example(A_star, B_star, qr_ratio, prime_horizon, prime_excitation)

def unstable_dynamics(qr_ratio=1e1, prime_horizon=250, prime_excitation=2):
    A_star, B_star = examples.transient_dynamics(diag_coeff=2, upperdiag=4)
    return set_up_example(A_star, B_star, qr_ratio, prime_horizon, prime_excitation, sigma_excitation=0.1)

In [10]:
example = laplacian_dynamics() # unstable_dynamics()
A_star, B_star, K_init, Q, R, prime_horizon, prime_excitation, sigma_excitation, sigma_w = example
prime_horizon = 1000

print(A_star)
print(B_star)
print(K_init)
print("prime_horizon", prime_horizon)
print("prime_excitation", prime_excitation)
print("sigma_excitation", sigma_excitation)

[[1.01 0.01 0.  ]
 [0.01 1.01 0.01]
 [0.   0.01 1.01]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[-0.04373095 -0.01250864 -0.00126936]
 [-0.01250864 -0.04500031 -0.01250864]
 [-0.00126936 -0.01250864 -0.04373095]]
prime_horizon 1000
prime_excitation 1
sigma_excitation 0.1


# Constructors for different adaptive methods

In [11]:
def optimal_ctor():
    return OptimalStrategy(Q=Q, R=R, A_star=A_star, B_star=B_star, sigma_w=sigma_w)

def nominal_ctor():
    return NominalStrategy(Q=Q,
                          R=R,
                          A_star=A_star,
                          B_star=B_star,
                          sigma_w=sigma_w,
                          sigma_explore=sigma_excitation,
                          reg=1e-5,
                          epoch_multiplier=10, rls_lam=None)

def ofu_ctor():
    return OFUStrategy(Q=Q,
                  R=R,
                  A_star=A_star,
                  B_star=B_star,
                  sigma_w=sigma_w,
                  reg=1e-5,
                  actual_error_multiplier=1, rls_lam=None)

def ts_ctor():
    return TSStrategy(Q=Q,
                  R=R,
                  A_star=A_star,
                  B_star=B_star,
                  sigma_w=sigma_w,
                  reg=1e-5,
                  tau=500,
                  actual_error_multiplier=1, rls_lam=None)

def sls_fir_ctor():
    return SLS_FIRStrategy(Q=Q,
                  R=R,
                  A_star=A_star,
                  B_star=B_star,
                  sigma_w=sigma_w,
                  sigma_explore=sigma_excitation,
                  reg=1e-5,
                  epoch_multiplier=10,
                  truncation_length=12,
                  actual_error_multiplier=1, rls_lam=None)

def sls_cl_ctor():
    return SLS_CommonLyapunovStrategy(Q=Q,
                  R=R,
                  A_star=A_star,
                  B_star=B_star,
                  sigma_w=sigma_w,
                  sigma_explore=sigma_excitation,
                  reg=1e-5,
                  epoch_multiplier=10,
                  actual_error_multiplier=1, rls_lam=None)

def lspi_ctor():
    return LSPIStrategy(Q=Q,
                        R=R,
                        A_star=A_star,
                        B_star=B_star,
                        sigma_w=sigma_w,
                        sigma_explore=sigma_excitation,
                        epoch_multiplier=10,
                        num_PI_iters=3,
                        K_init=K_init)
def mflq_ctor():
    return MFLQStrategy(Q=Q,
                        R=R,
                        A_star=A_star,
                        B_star=B_star,
                        sigma_w=sigma_w,
                        sigma_explore=sigma_excitation,
                        epoch_length=0.5 * horizon ** (3.0/ 4),
                        exploration_period=horizon ** (1.0 / 4),
                        K_init=K_init)

# Helper methods for running in parallel

In [12]:
prime_seed = 45727
def run_one_trial(new_env_ctor, seed, prime_fixed=False):
    rng = np.random.RandomState(seed)
    if prime_fixed: # reducing variance
        rng_prime = np.random.RandomState(prime_seed) 
    else:
        rng_prime = rng
    env = new_env_ctor()
    env.reset(rng_prime)
    env.prime(prime_horizon, K_init, prime_excitation, rng_prime)
    regret = np.array([env.step(rng) for _ in range(horizon)])
    env.complete_epoch(rng)
    err, cost = env.get_statistics(iteration_based=True)
    return regret, err, cost

def spawn_invocation(method, p, prime_fixed=False):
    seed = np.random.randint(0xFFFFFFFF)
    ctor = {
        'optimal': optimal_ctor,
        'nominal': nominal_ctor,
        'ofu': ofu_ctor,
        'ts': ts_ctor,
        'sls_fir': sls_fir_ctor,
        'sls_cl': sls_cl_ctor,
        'lspi': lspi_ctor,
        'mflq': mflq_ctor,
    }[method]
    return (p.apply_async(run_one_trial, (ctor, seed, prime_fixed)), seed)

def process_future_list(ftchs):
    regrets = []
    errors = []
    costs = []
    seeds = []
    bad_invocations = 0
    for ftch, seed in ftchs:
        try:
            reg, err, cost = ftch.get()
        except Exception as e:
            traceback.print_exc()
            bad_invocations += 1
            continue
        regrets.append(reg)
        errors.append(err)
        costs.append(cost)
        seeds.append(seed)
    return np.array(regrets), np.array(errors), np.array(costs), np.array(seeds), bad_invocations

# Running experiments and plotting results

In [13]:
#strategies = ['optimal', 'nominal', 'ofu', 'ts', 'sls_fir', 'lspi']
strategies = ['mflq'] #['optimal', 'nominal', 'lspi', 'mflq']
#strategies = ['optimal', 'nominal']
start_time = time.time()
with Pool(processes=cpu_count()) as p:
    all_futures = [[spawn_invocation(method, p, prime_fixed=True) 
                    for _ in range(trials_per_method)] for method in strategies]
    list_of_results = [process_future_list(ftchs) for ftchs in all_futures] 
print("finished execution in {} seconds".format(time.time() - start_time))

INFO:mflq:_design_controller(epoch=0): n_transitions=1000
INFO:mflq:_design_controller(epoch=0): n_transitions=1000
INFO:mflq:_design_controller(epoch=0): n_transitions=1000
INFO:mflq:_design_controller(epoch=0): n_transitions=1000
INFO:mflq:_design_controller(epoch=0): n_transitions=1000
INFO:mflq:_design_controller(epoch=0): n_transitions=1000


phis shape (1000, 9)
phis shape (1000, 9)
phis 6
Psis shape (1000, 36)
phis 6
phis shape (1000, 9)
Psis shape (1000, 36)
phis shape (1000, 9)
phis 6
Psis shape (1000, 36)
phis 6
Psis shape (1000, 36)
phis shape (1000, 9)
phis 6
Psis shape (1000, 36)
Psis 21
Psis 21


../python/mflq.py:128: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  hhat = np.linalg.lstsq(Amat, bmat)[0]


Psis 21


../python/mflq.py:128: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  hhat = np.linalg.lstsq(Amat, bmat)[0]
../python/mflq.py:128: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  hhat = np.linalg.lstsq(Amat, bmat)[0]


...

multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/home/karlk/bin/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-12-2e9edc92baef>", line 10, in run_one_trial
    env.prime(prime_horizon, K_init, prime_excitation, rng_prime)
  File "../python/adaptive.py", line 168, in prime
    rng)
  File "../python/mflq.py", line 113, in _design_controller
    self._sigma_w, n)
  File "../python/mflq.py", line 132, in _estimate_G
    ghat = scipy.linalg.solve(Psis.T @ Psis, Psis.T @ (costs + (Phis_plus - W) @ hhat_proj), sym_pos=True)
  File "/home/karlk/bin/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py", line 250, in solve
    _solve_check(n, info)
  File "/home/karlk/bin/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py", line 31, in _solve_check
    raise LinAlgError('Matrix is singular.')
numpy.linalg.linalg.LinAlgError: Matrix is singular.
"""

The ab

Psis 21
Psis 21


../python/mflq.py:128: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  hhat = np.linalg.lstsq(Amat, bmat)[0]


finished execution in 1.9454333782196045 seconds


multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/home/karlk/bin/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-12-2e9edc92baef>", line 10, in run_one_trial
    env.prime(prime_horizon, K_init, prime_excitation, rng_prime)
  File "../python/adaptive.py", line 168, in prime
    rng)
  File "../python/mflq.py", line 113, in _design_controller
    self._sigma_w, n)
  File "../python/mflq.py", line 132, in _estimate_G
    ghat = scipy.linalg.solve(Psis.T @ Psis, Psis.T @ (costs + (Phis_plus - W) @ hhat_proj), sym_pos=True)
  File "/home/karlk/bin/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py", line 250, in solve
    _solve_check(n, info)
  File "/home/karlk/bin/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py", line 31, in _solve_check
    raise LinAlgError('Matrix is singular.')
numpy.linalg.linalg.LinAlgError: Matrix is singular.
"""

The ab

In [1]:
trials_per_method

NameError: name 'trials_per_method' is not defined

In [2]:
def get_errorbars(regrets, q=10, percent_bad=0):
    print(percent_bad)
    median = np.percentile(regrets, q=50-percent_bad, axis=0)
    low10 = np.percentile(regrets, q=q, axis=0)
    high90 = np.percentile(regrets, q=100-(q-percent_bad), axis=0)
    return median, low10, high90

def plot_list_medquantile(datalist, legendlist=None, xlabel=None, ylabel=None, semilogy=False, 
                          loc='upper left', alpha=0.1, figsize=(8,4)):
    rgblist = sns.color_palette('viridis', len(datalist))
    plt.figure(figsize=figsize)
    for idx, data in enumerate(datalist):
        median, lower, higher = data
        if semilogy:
            plt.semilogy(range(len(median)), median, color=rgblist[idx], label=legendlist[idx])
        else:
            plt.plot(range(len(median)), median, color=rgblist[idx], label=legendlist[idx])
        plt.fill_between(np.array(np.arange(len(median))), median.astype(np.float), 
                        higher.astype(np.float), color=rgblist[idx], alpha=alpha)
    if legendlist is not None:
        plt.legend(loc=loc)
    if xlabel is not None:
        plt.xlabel(xlabel)
    if ylabel is not None:
        plt.ylabel(ylabel)

In [3]:
regretlist = []
costs_list = []

#strat_rearranged =  [strategies[2], strategies[3], strategies[1], strategies[0], strategies[4], strategies[5]]
#res_rearranged =  [list_of_results[2], list_of_results[3], list_of_results[1], list_of_results[0], list_of_results[4], list_of_results[5]]

strat_rearranged = strategies
res_rearranged = list_of_results

for name, result in zip(strat_rearranged, res_rearranged):
    regrets, errors, costs, _, bad_invocations = result
    print(name, "bad_invocations", bad_invocations)
    percent_bad = bad_invocations / trials_per_method * 100
    regretlist.append(get_errorbars(regrets, q=10, percent_bad=percent_bad))
    costs_list.append(get_errorbars(costs, q=10, percent_bad=percent_bad))

sns.set_palette("muted")
plot_list_medquantile(regretlist, legendlist=strat_rearranged, xlabel="Iteration", ylabel="Regret")
plot_list_medquantile(costs_list, legendlist=strat_rearranged, xlabel="Iteration", 
                      ylabel="Cost Suboptimality", semilogy=True, loc='upper right')


NameError: name 'strategies' is not defined

In [4]:
costs_list

[]

In [5]:
run_one_trial(lspi_ctor, 123, True)

NameError: name 'run_one_trial' is not defined

In [6]:
run_one_trial(nominal_ctor, 123, True)

NameError: name 'run_one_trial' is not defined